In [32]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import datetime
from datetime import datetime
from os import path, environ

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipyvolume as ipv

from preprocess import POW_FIELDS, TEMP_FIELDS
from thermo import CONSTANTS
# source file, see docs/5-dataset.md for info on field names
chiller_file = path.join(environ['DATADIR'], 'EngineeringScienceBuilding', 'Chillers.csv')
plot_path = path.join('..', 'docs', 'img')

# Trends

## Dataset

The dataset describes input/output powers, temperatures, and cooling by various components of a chiller with a water-cooled condenser and a cooling tower. More details are in `/docs/`.

After pre-processing, all measurements are in SI units (Kelvins, watts) except for the field `KWPerTon` where units are explicit in the measurement.

In [28]:
# Read pre-processed data
df = pd.read_csv(chiller_file, index_col='Time', parse_dates=True, dtype=float)
df.head()

,FlowEvap,FreqFanA,FreqFanB,PerChiLoad,PerFreqFanA,PerFreqFanB,PerHumidity,PowChi,PowChiP,PowConP,PowCool,PowFanA,PowFanB,PowIn,TempAmbient,TempCondIn,TempCondOut,TempEvapIn,TempEvapOut,TempWetBulb
Time,,,,,,,,,,,,,,,,,,,,
2018-03-20 00:05:00,0.001838,59.94,59.40,20.26248,0.991757,0.991757,0.850263,92400.0,17270.0,1830.0,10986.817135,9849.999,9440.000,130789.999,287.673818,290.27,296.91,280.02,278.59,285.974983
2018-03-20 00:15:00,0.001843,39.03,38.53,18.91911,0.991757,0.991757,0.844437,90800.0,17280.0,1500.0,10786.147037,3290.000,3110.000,115980.000,287.509980,292.06,296.29,280.14,278.74,285.751533
2018-03-20 00:20:00,0.001829,59.94,59.40,15.25515,0.991757,0.991757,0.852229,88200.0,17130.0,2860.0,8868.208741,9650.000,9139.999,126979.999,287.499097,289.19,296.13,279.62,278.46,285.826600
2018-03-20 00:30:00,0.001835,59.94,59.40,17.36743,0.991757,0.991757,0.857587,91200.0,17400.0,2020.0,9588.182507,9879.999,9709.999,130209.998,287.389079,292.02,296.97,279.79,278.54,285.778294
2018-03-20 00:35:00,0.001836,59.94,59.40,17.12868,0.991757,0.991757,0.859493,86900.0,17200.0,1080.0,10438.696572,9700.000,9490.000,124370.000,287.343824,288.17,295.77,280.13,278.77,285.755078


## Efficiency metrics

### Coefficient of performance

$$
\begin{align*}
COP_{max cooling} &= \frac{T_{cold}}{T_{hot} - T_{cold}}  \\
COP &= \frac{\texttt{Energy Extracted}}{\texttt{Energy Input}}
\end{align*}
$$

In [29]:
# Measure maximum Coefficient of Performance (COP) for cooling
# for the ENTIRE chiller plant.
# Currently in the dataset, water flow rate for the condenser loop
# is not available - so COP for condenser/cooling tower cannot be
# calculated.
df['COPMax'] = df['TempEvapIn'] / (df['TempEvapIn'] - df['TempEvapOut'])

# Measured achieved COP
df['COP'] = df['PowCool'] / df['PowIn']

In [30]:
# Aggregating measurements across days by time of day
downsampled = df.asfreq('15T')                       # downsampling readings for cleaner plot
bytime = downsampled.groupby(downsampled.index.time) # group all records by time of day
means = bytime.mean()                                # calculate average values across days
means.set_index(pd.to_datetime(means.index, format='%H:%M:%S').time, inplace=True)

## Plots

### COP

In [ ]:
f = plt.figure(figsize=(12, 6))
f.suptitle('COP (Entire plant) - Daily Averages')

ax = means['COP'].plot(legend=True, logy=True, yerr=bytime['COP'].std())
ax = means['COPMax'].plot(legend=True, logy=True, yerr=bytime['COPMax'].std())
ax.set(ylabel='COP');
plt.savefig(path.join(plot_path, '6-COP.png'))

### Temperatures

In [ ]:
f = plt.figure(figsize=(12, 8))
f.suptitle('Temperature - Daily Averages')

for col in TEMP_FIELDS:
    ax = means[col].plot(legend=True)
ax.set(ylabel='Temperature (K)');
plt.savefig(path.join(plot_path, '6-temps.png'))

### Power vs. Cooling

Input power vs. total cooling done by the evaporator.

In [ ]:
f = plt.figure(figsize=(12, 8))
f.suptitle('Input power vs. Cooling')

ax = means['PowIn'].plot(legend=True, yerr=bytime['PowIn'].std())
ax = means['PowCool'].plot(legend=True, yerr=bytime['PowCool'].std())

ax.set(ylabel='Power (watts)')
ax.set_ylim(bottom=0)
plt.savefig(path.join(plot_path, '6-pow-vs-cooling.png'))

### Condenser/cooling tower cycle power

In [ ]:
f = plt.figure(figsize=(12, 8))
f.suptitle('Cooling tower power vs cooling')

ax = means['PowFanA'].plot(legend=True)
ax = means['PowFanB'].plot(legend=True)
ax = means['PowConP'].plot(legend=True)
ax.set(ylabel='Power (watts)')
ax = means['TempCondOut'].plot(legend=True, secondary_y=True, linestyle='--')
ax = means['TempCondIn'].plot(legend=True, secondary_y=True, linestyle='--')
ax = means['TempAmbient'].plot(legend=True, secondary_y=True, linestyle='--')
ax = means['TempWetBulb'].plot(legend=True, secondary_y=True, linestyle='--')
ax.set(ylabel='Temp (K)')
ax.set_ylim(bottom=273.15)  # starting at freezing
plt.savefig(path.join(plot_path, '6-cooling-tower-pow.png'))

### Fan speed (%) vs fan power

In [ ]:
f = plt.figure(figsize=(12, 8))
f.suptitle('Fan power (w) and fan speed (%)')

ax = means['PowFanA'].plot(legend=True)
ax = means['PowFanB'].plot(legend=True)
ax = means.loc[:, ('PowFanA', 'PowFanB')].mean(axis=1)\
          .rolling(window=4, center=True).mean()\
          .plot(legend=True, label='Average Power', linewidth=3)
ax.set(ylabel='Power (watts)')
ax = means['PerFreqFanA'].plot(legend=True, secondary_y=True, linestyle='--')
ax = means['PerFreqFanB'].plot(legend=True, secondary_y=True, linestyle='--')
ax.set(ylabel='Percentage speed (%)')
ax.set_ylim(bottom=0);
plt.savefig(path.join(plot_path, '6-fan-power-speed.png'))